## Imports

In [175]:
import pytesseract
from pdf2image import convert_from_path
import os
import re

import pandas as pd
# Set the maximum number of columns to None (unlimited)
pd.set_option('display.max_columns', None)

import numpy as np
import zipfile
from PIL import Image, ImageFilter, ImageOps
import io
import csv
from thefuzz import process # Install via: pip install thefuzz
import glob
import pickle
import multiprocessing as mp

In [181]:
# --- PATHS ---
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
poppler_bin_path = r'C:\poppler\poppler-25.12.0\Library\bin' 
input_file = r'C:\pdf2csv\extracted_files\recovered\6852.pdf'

# High Quality

In [ ]:
try:
    print("1. Converting PDF to high-resolution image...")
    images = convert_from_path(
        input_file,
        dpi=400,
        first_page=1,
        last_page=1,
        poppler_path=poppler_bin_path
    )

    print("2. Preprocessing image for OCR...")
    img = images[0]

    img = img.convert("L")
    img = ImageOps.autocontrast(img)
    img = img.filter(ImageFilter.MedianFilter(size=3))
    img = img.point(lambda x: 0 if x < 160 else 255, "1")

    print("3. Running OCR (optimized for scanned documents)...")
    custom_config = r'-l eng --oem 3 --psm 4 -c preserve_interword_spaces=1'
    text = pytesseract.image_to_string(img, config=custom_config)

    print("4. Cleaning OCR text...")
    lines = [line.strip() for line in text.splitlines() if line.strip()]

    data = []
    for line in lines:
        line = line.replace(" ,", ",").replace(", ", ",")
        data.append(line.split(","))

    print("5. Writing CSV...")
    df = pd.DataFrame(data)

except Exception as e:
    print(f"Error: {e}")
    
data

In [ ]:
data[1]

In [ ]:
def extract_ocr_data(data_list):
    # 1. Clean and Join: Remove noise characters but keep structure
    raw_text = " ".join(data_list)
    # Remove common OCR artifacts that break numbers
    clean_text = re.sub(r'[“”‘’"°*]', '', raw_text)
    parts = clean_text.split()
    
    # 2. Extract Dates (Handling formats like 1/22/1993, 10.23.2002, or split 12 23 1992)
    # This finds 4-digit years and looks backwards for the month/day
    dates = []
    date_pattern = r'(\d{1,2}[/.-]\d{1,2}[/.-]\d{4}|\d{1,2}[/.-]\d{4})'
    found_dates = re.findall(date_pattern, clean_text)
    
    # If dates are split by spaces (e.g., '12', '23', '1992'), we reconstruct them
    if not found_dates:
        for i in range(len(parts)):
            if len(parts[i]) == 4 and parts[i].isdigit(): # Found a year
                if i >= 2 and parts[i-1].isdigit() and parts[i-2].isdigit():
                    found_dates.append(f"{parts[i-2]}/{parts[i-1]}/{parts[i]}")
    
    # 3. Dynamic Field Extraction
    
    # Lender: Usually the first 2-3 words
    lender = " ".join([p for p in parts[:3] if not any(c.isdigit() for c in p)])
    
    # Status: Find specific keywords
    es_status = next((p for p in parts if "Terminated" in p), "Unknown")
    
    # Numbers: Find all decimals and integers
    numbers = re.findall(r'\d+\.\d+|\d+', clean_text)
    
    # Ages: Usually 2-digit numbers between 62 and 99
    ages = [n for n in numbers if 62 <= float(n) <= 100 and len(n) == 2]
    borr_age = ages[0] if len(ages) > 0 else "no data"
    coborr_age = ages[1] if len(ages) > 1 else "no data"
    
    # Interest Rate: Usually a decimal like 7.99 or 8.27
    rates = [n for n in numbers if "." in n and 2 <= float(n) <= 15]
    int_rt = rates[0] if rates else "0.00"
    
    # Zip Code: 5 digit number
    zip_code = [n for n in numbers if len(n) == 5]
    prop_zip = zip_code[0] if zip_code else "00000"
    
    # Financial Amounts: Large numbers (Claim Amount and Principal Limit)
    amounts = [n for n in numbers if float(n) > 5000 and len(n) > 4]
    max_clm = amounts[0] if len(amounts) > 0 else "0"
    init_limit = amounts[1] if len(amounts) > 1 else "0"

    # Flag Extraction (Searching for specific characters N, A, T)
    pd_stmln = "N" if "N" in clean_text.upper() else "Y"
    rt_typ = "A" if "A" in clean_text.upper() else "F"
    arm_indx = "T" if "T" in clean_text.upper() else "N"

    # 4. Map to your exact list structure
    mapped_output = [
        lender,
        found_dates[0] if len(found_dates) > 0 else "no data",
        found_dates[1] if len(found_dates) > 1 else "no data",
        found_dates[2] if len(found_dates) > 2 else "no data",
        borr_age,
        coborr_age,
        "1", # Default count
        es_status,
        int_rt,
        "0.0", # int_rt_10yr
        "0.0", # hecm_margin
        pd_stmln,
        rt_typ,
        arm_indx,
        "1Y", # arm_prdc_typ
        max_clm,
        init_limit,
        "0", # fees
        prop_zip,
        "02" # loan_typ
    ]
    
    return ", ".join(map(str, mapped_output))

# Test with one of your problematic rows
row = ['UNETY', 'MORTGAGE', 'CORP', '12', '23', '1992', '3.25', '1994', '10', '15', '1999', '81', '74', '20', '8.39', 'H.', '39', 'BN', '"A', '1', 'Ww', 'Loz700', '60490.', '5', 'on']
print(extract_ocr_data(row))

# Low Quality

In [189]:
try:
    print("1. Converting PDF to Image...")
    images = convert_from_path(input_file, first_page=1, last_page=1, poppler_path=poppler_bin_path)
    
    print("2. Running OCR with Table Segmentation...")
    # --psm 6 tells Tesseract to treat the image as a single uniform block of text
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(images[0], config=custom_config)

except Exception as e:
    print(f"\nError: {e}")

1. Converting PDF to Image...
2. Running OCR with Table Segmentation...


In [188]:
text

'("BANK OF AMERICA NA CHARLOTTE’ 2/5/2010] _3/s/aonof | 5] sof |Endorsed @T2.970f Pfu ae 23z0007 7206] sooo [97741 "02"\n[|METLIFE BANK, NATIONAL aSsocrATION’ | 1720/2010] 2/aa/aono[ eo] Assigned [5.56] 56] fins eT to6000/2asaza| 5000 [970 "02" |\n["BANK OF AMERICA NA CHARLOTTE’ | 2/5/2010] 3/4/2010] 12/ao/eors| 66] | | Terminated’ | 2.970) 63a] rs fut ae 87o00| 92565] azo o7a1a "02"\n[TRINANCIAL FREEDOM ACQUISITION LLC’ | 2/2/2010] s/i/ao10[77as/aor9| | | Terminatea [5.56] 56] fn 220000136400 4200 |" 97702 "02" |\n[LAND HOME FINANCIAL SERVICES, INC.’ | 3/26/2010] _a/a6/2oi0| | so] | Assigned ‘| 5.56) 5.56] oft’ th 2500] s69670.5] sooo 97110 * "02"\n[GENERATION MORTGAGE CowpANY’ | 2/7/2010] 3/9/2010|9/is/aoia| || | "Terminated’ | 48299] as ftNh wim 20000] azs6z0[az00["97ia1 "02"\n[TFINANCIAL FREEDOM ACQUISITION LLC’ | 2/22/2010] s/ig/2oi0| | sa] | Assigned \' | 5.56) 5.56] oft th 320000 tzasto] gg99 [97132 "on"\n[METLIFE BANK, NATIONAL AssocraTion’ | a/5/2010|2/aa/a0i0| | wat Assigned 

In [190]:
target_columns = [
    'lender', 'clsing_dt', 'endrsmt_dt', 'tmntn_dt', 'Borr_Age', 
    'Coborr_Age', 'Borr_Cnt', 'es_status', 'int_rt', 'int_rt_10yr', 
    'hecm_margin', 'pd_stmln_flg', 'rt_typ', 'arm_indx_typ', 
    'arm_prdc_typ', 'max_clm_amt', 'init_prncpl_lmt', 
    'hecm_orgntn_fees', 'prop_addr_zip_cd', 'loan_typ'
]

def robust_parse(line):
    row = [None] * 20

    # ---------- OCR NORMALIZATION ----------
    line = (
        line.replace('”', '"')
            .replace("’", "'")
            .replace('!', '')
            .replace('|', ' ')
    )

    # ---------- PATTERN EXTRACTION ----------
    dates = re.findall(r'\d{1,2}/\d{1,2}/\d{2,4}', line)
    decimals = re.findall(r'\d+\.\d+', line)

    # Remove dates & decimals before integer parsing
    clean_text = re.sub(r'\d{1,2}/\d{1,2}/\d{2,4}', ' ', line)
    clean_text = re.sub(r'\d+\.\d+', ' ', clean_text)
    numbers = re.findall(r'\b\d+\b', clean_text)

    # ---------- LENDER ----------
    lender_match = re.search(r'^(.*?)(?=\d{1,2}/)', line)
    row[0] = lender_match.group(1).strip().strip('"').strip("'") if lender_match else None

    # ---------- DATES ----------
    if len(dates) >= 1: row[1] = dates[0]
    if len(dates) >= 2: row[2] = dates[1]
    if len(dates) >= 3: row[3] = dates[2]

    # ---------- STATUS ----------
    if re.search(r'Termin[a-z]*', line, re.I):
        row[7] = "Terminated"
    elif re.search(r'Active', line, re.I):
        row[7] = "Active"

    # ---------- DECIMALS ----------
    if len(decimals) >= 1: row[8] = decimals[0]
    if len(decimals) >= 2: row[9] = decimals[1]
    if len(decimals) >= 3: row[10] = decimals[2]

    # ---------- AGES & COUNT (DOMAIN RULE APPLIED) ----------
    if row[3] and row[7]:
        try:
            start = line.index(row[3]) + len(row[3])
            end = line.lower().index(row[7].lower())
            block = line[start:end]
            nums = re.findall(r'\b\d+\b', block)

            if len(nums) == 3:
                row[4], row[5], row[6] = nums
            elif len(nums) == 2:
                row[4], row[6] = nums
                row[5] = None
        except ValueError:
            pass

    # ---------- FLAGS ----------
    flags = re.findall(r'\b[A-Z]\b', line.upper())
    if len(flags) >= 1: row[11] = flags[0]
    if len(flags) >= 2: row[12] = flags[1]
    if len(flags) >= 3: row[13] = flags[2]

    # ---------- ARM PRODUCT ----------
    arm_match = re.search(r'\b\d+[A-Z]\b', line)
    row[14] = arm_match.group(0) if arm_match else None

    # ---------- MAX CLAIM & PRINCIPAL ----------
    large_nums = [n for n in numbers if len(n) > 3]

    if len(large_nums) >= 1:
        row[15] = large_nums[0]      # max_clm_amt
    if len(large_nums) >= 2:
        row[16] = large_nums[1]      # init_prncpl_lmt

    # ---------- FEES ----------
    row[17] = 0

    # ---------- ZIP & LOAN TYPE (RESTORED LOGIC) ----------
    if len(numbers) >= 2:
        row[19] = str(numbers[-1]).zfill(2)   # loan_typ
        row[18] = numbers[-2] if len(numbers[-2]) == 5 else None

    return row

# ---------- BUILD DATAFRAME ----------
lines = text.split('\n')
rows = [robust_parse(l) for l in lines if len(l.strip()) > 20]

df = pd.DataFrame(rows, columns=target_columns)

df.tail(30)

,lender,clsing_dt,endrsmt_dt,tmntn_dt,Borr_Age,Coborr_Age,Borr_Cnt,es_status,int_rt,int_rt_10yr,hecm_margin,pd_stmln_flg,rt_typ,arm_indx_typ,arm_prdc_typ,max_clm_amt,init_prncpl_lmt,hecm_orgntn_fees,prop_addr_zip_cd,loan_typ
35,"[""PLAZA HOME MORTGAGE, INC.' a/tovaoi0 3",None,None,None,None,None,None,None,5.56,None,None,A,None,None,None,3660,97459,0,97459,02
36,[WELLS FARGO BANK NA' 32010],4/5/2010,5/16/2017,None,None,None,None,Terminated,5.49,None,None,None,None,None,None,32010,5000,0,97138,02
37,"[""BANK OF AMERICA NA CHARLOTTE",4/19/2010,None,None,None,None,None,Terminated,5.56,None,None,None,None,None,None,97420,None,0,97420,02
38,"[""PINANCTAL FREEDOM ACQUISITION LLC",3/19/2010,None,None,None,None,None,None,2.482,None,None,None,None,None,None,00000,None,0,None,00000
39,[WELLS FARGO BANK NAY aafaono,3/12/2010,None,None,None,None,None,Terminated,5.49,None,None,A,A,None,None,30000,97381,0,97381,02
40,[WELLS FARGO BANK NAT a/ayaoi0,None,None,None,None,None,None,None,None,None,None,A,None,None,None,97603,None,0,97603,02
41,['BANK OF AMERICA NA cuaRLorTE,4/15/2010,None,None,None,None,None,None,2.720,6.08,None,None,None,None,None,2010,97071,0,97071,02
42,[JAMES B NUTTER AND ComPaNY' a/a6 2010],2/12/2010,None,None,None,None,None,Terminated,None,None,None,B,A,None,None,2010,37000,0,97386,02
43,['BANK OF AMERICA NA CHARLOTTE,None,None,None,None,None,None,Terminated,5.56,5.6,None,O,S,None,None,2010,5000,0,None,02
44,[WELLS FARGO BANK NA a/azaono,None,None,None,None,None,None,None,None,None,None,A,A,None,None,310000,97064,0,97064,02


In [109]:
%reset
%whos function

No variables match your requested type.


# Final

In [ ]:
input_file = r'C:\pdf2csv\extracted_files\recovered\22.pdf'

print("1. Converting PDF to Image...")
images = convert_from_path(input_file, first_page=1, last_page=1, poppler_path=poppler_bin_path)

print("2. Running OCR with Table Segmentation...")
# --psm 6 tells Tesseract to treat the image as a single uniform block of text
custom_config = r'--oem 3 --psm 6'
text = pytesseract.image_to_string(images[0], config=custom_config)

target_columns = [
    'lender', 'clsing_dt', 'endrsmt_dt', 'tmntn_dt', 'Borr_Age', 
    'Coborr_Age', 'Borr_Cnt', 'es_status', 'int_rt', 'int_rt_10yr', 
    'hecm_margin', 'pd_stmln_flg', 'rt_typ', 'arm_indx_typ', 
    'arm_prdc_typ', 'max_clm_amt', 'init_prncpl_lmt', 
    'hecm_orgntn_fees', 'prop_addr_zip_cd', 'loan_typ'
]

def robust_parse(line):
    row = [None] * 20

    # ---------- OCR NORMALIZATION ----------
    line = (
        line.replace('”', '"')
            .replace("’", "'")
            .replace('!', '')
            .replace('|', ' ')
    )

    # ---------- PATTERN EXTRACTION ----------
    dates = re.findall(r'\d{1,2}/\d{1,2}/\d{2,4}', line)
    decimals = re.findall(r'\d+\.\d+', line)

    # Remove dates & decimals before integer parsing
    clean_text = re.sub(r'\d{1,2}/\d{1,2}/\d{2,4}', ' ', line)
    clean_text = re.sub(r'\d+\.\d+', ' ', clean_text)
    numbers = re.findall(r'\b\d+\b', clean_text)

    # ---------- LENDER ----------
    lender_match = re.search(r'^(.*?)(?=\d{1,2}/)', line)
    row[0] = lender_match.group(1).strip().strip('"').strip("'") if lender_match else None

    # ---------- DATES ----------
    if len(dates) >= 1: row[1] = dates[0]
    if len(dates) >= 2: row[2] = dates[1]
    if len(dates) >= 3: row[3] = dates[2]

    # ---------- STATUS ----------
    if re.search(r'Termin[a-z]*', line, re.I):
        row[7] = "Terminated"
    elif re.search(r'Active', line, re.I):
        row[7] = "Active"

    # ---------- DECIMALS ----------
    if len(decimals) >= 1: row[8] = decimals[0]
    if len(decimals) >= 2: row[9] = decimals[1]
    if len(decimals) >= 3: row[10] = decimals[2]

    # ---------- AGES & COUNT (DOMAIN RULE APPLIED) ----------
    if row[3] and row[7]:
        try:
            start = line.index(row[3]) + len(row[3])
            end = line.lower().index(row[7].lower())
            block = line[start:end]
            nums = re.findall(r'\b\d+\b', block)

            if len(nums) == 3:
                row[4], row[5], row[6] = nums
            elif len(nums) == 2:
                row[4], row[6] = nums
                row[5] = None
        except ValueError:
            pass

    # ---------- FLAGS ----------
    flags = re.findall(r'\b[A-Z]\b', line.upper())
    if len(flags) >= 1: row[11] = flags[0]
    if len(flags) >= 2: row[12] = flags[1]
    if len(flags) >= 3: row[13] = flags[2]

    # ---------- ARM PRODUCT ----------
    arm_match = re.search(r'\b\d+[A-Z]\b', line)
    row[14] = arm_match.group(0) if arm_match else None

    # ---------- MAX CLAIM & PRINCIPAL ----------
    large_nums = [n for n in numbers if len(n) > 3]

    if len(large_nums) >= 1:
        row[15] = large_nums[0]      # max_clm_amt
    if len(large_nums) >= 2:
        row[16] = large_nums[1]      # init_prncpl_lmt

    # ---------- FEES ----------
    row[17] = 0

    # ---------- ZIP & LOAN TYPE (RESTORED LOGIC) ----------
    if len(numbers) >= 2:
        row[19] = str(numbers[-1]).zfill(2)   # loan_typ
        row[18] = numbers[-2] if len(numbers[-2]) == 5 else None

    return row

# ---------- BUILD DATAFRAME ----------
lines = text.split('\n')
rows = [robust_parse(l) for l in lines if len(l.strip()) > 20]

df = pd.DataFrame(rows, columns=target_columns)

In [112]:
input_file = r'C:\pdf2csv\extracted_files\recovered\22.pdf'

print("1. Converting PDF to Image...")
images = convert_from_path(input_file, first_page=1, last_page=1, poppler_path=poppler_bin_path)

print("2. Running OCR with Table Segmentation...")
# --psm 6 tells Tesseract to treat the image as a single uniform block of text
custom_config = r'--oem 3 --psm 6'
text = pytesseract.image_to_string(images[0], config=custom_config)

target_columns = [
    'lender', 'clsing_dt', 'endrsmt_dt', 'tmntn_dt', 'Borr_Age', 
    'Coborr_Age', 'Borr_Cnt', 'es_status', 'int_rt', 'int_rt_10yr', 
    'hecm_margin', 'pd_stmln_flg', 'rt_typ', 'arm_indx_typ', 
    'arm_prdc_typ', 'max_clm_amt', 'init_prncpl_lmt', 
    'hecm_orgntn_fees', 'prop_addr_zip_cd', 'loan_typ'
]

def robust_parse(line):
    row = [None] * 20

    # ---------- OCR NORMALIZATION ----------
    line = (
        line.replace('”', '"')
            .replace("’", "'")
            .replace('!', '')
            .replace('|', ' ')
    )

    # ---------- PATTERN EXTRACTION ----------
    dates = re.findall(r'\d{1,2}/\d{1,2}/\d{2,4}', line)
    decimals = re.findall(r'\d+\.\d+', line)

    # Remove dates & decimals before integer parsing
    clean_text = re.sub(r'\d{1,2}/\d{1,2}/\d{2,4}', ' ', line)
    clean_text = re.sub(r'\d+\.\d+', ' ', clean_text)
    numbers = re.findall(r'\b\d+\b', clean_text)

    # ---------- LENDER ----------
    lender_match = re.search(r'^(.*?)(?=\d{1,2}/)', line)
    row[0] = lender_match.group(1).strip().strip('"').strip("'") if lender_match else None

    # ---------- DATES ----------
    if len(dates) >= 1: row[1] = dates[0]
    if len(dates) >= 2: row[2] = dates[1]
    if len(dates) >= 3: row[3] = dates[2]

    # ---------- STATUS ----------
    if re.search(r'Termin[a-z]*', line, re.I):
        row[7] = "Terminated"
    elif re.search(r'Active', line, re.I):
        row[7] = "Active"

    # ---------- DECIMALS ----------
    if len(decimals) >= 1: row[8] = decimals[0]
    if len(decimals) >= 2: row[9] = decimals[1]
    if len(decimals) >= 3: row[10] = decimals[2]

    # ---------- AGES & COUNT (DOMAIN RULE APPLIED) ----------
    if row[3] and row[7]:
        try:
            start = line.index(row[3]) + len(row[3])
            end = line.lower().index(row[7].lower())
            block = line[start:end]
            nums = re.findall(r'\b\d+\b', block)

            if len(nums) == 3:
                row[4], row[5], row[6] = nums
            elif len(nums) == 2:
                row[4], row[6] = nums
                row[5] = None
        except ValueError:
            pass

    # ---------- FLAGS ----------
    flags = re.findall(r'\b[A-Z]\b', line.upper())
    if len(flags) >= 1: row[11] = flags[0]
    if len(flags) >= 2: row[12] = flags[1]
    if len(flags) >= 3: row[13] = flags[2]

    # ---------- ARM PRODUCT ----------
    arm_match = re.search(r'\b\d+[A-Z]\b', line)
    row[14] = arm_match.group(0) if arm_match else None

    # ---------- MAX CLAIM & PRINCIPAL ----------
    large_nums = [n for n in numbers if len(n) > 3]

    if len(large_nums) >= 1:
        row[15] = large_nums[0]      # max_clm_amt
    if len(large_nums) >= 2:
        row[16] = large_nums[1]      # init_prncpl_lmt

    # ---------- FEES ----------
    row[17] = 0

    # ---------- ZIP & LOAN TYPE (RESTORED LOGIC) ----------
    if len(numbers) >= 2:
        row[19] = str(numbers[-1]).zfill(2)   # loan_typ
        row[18] = numbers[-2] if len(numbers[-2]) == 5 else None

    return row

# ---------- BUILD DATAFRAME ----------
lines = text.split('\n')
rows = [robust_parse(l) for l in lines if len(l.strip()) > 20]

df = pd.DataFrame(rows, columns=target_columns)

# ---------- ADD SOURCE METADATA COLUMNS ----------
source_pdf = input_file.split('\\')[-1].replace('.pdf', '')
df.insert(0, 'row_in_pdf', range(1, len(df) + 1))
df.insert(0, 'source_pdf', source_pdf)

df

1. Converting PDF to Image...
2. Running OCR with Table Segmentation...


,source_pdf,row_in_pdf,lender,clsing_dt,endrsmt_dt,tmntn_dt,Borr_Age,Coborr_Age,Borr_Cnt,es_status,...,hecm_margin,pd_stmln_flg,rt_typ,arm_indx_typ,arm_prdc_typ,max_clm_amt,init_prncpl_lmt,hecm_orgntn_fees,prop_addr_zip_cd,loan_typ
0,22,1,“JAMES NUTTER AND COMPANY,6/15/2007,7/27/2007,4/21/2016,73,None,1,Terminated,...,None,T,None,None,None,88000,60808,0,35005,02
1,22,2,“JAMES B NUTTER AND COMPANY,6/19/2007,7/24/2007,4/6/2009,98,None,1,Terminated,...,None,B,T,None,None,108000,96444,0,36037,02
2,22,3,WELLS FARGO BANK NA,8/21/2007,11/20/2007,11/4/2019,72,None,1,Terminated,...,None,V,None,None,None,200000,138200,0,None,02
3,22,4,PACIFIC REVERSE MORTGAGE INC,1/18/2007,8/10/2007,10/29/2008,None,None,None,Terminated,...,1.5,U,N,T,None,130000,85410,0,85410,02
4,22,5,WELLS FARGO BANK NA,6/25/2007,7/18/2007,None,None,None,None,None,...,None,T,Y,None,None,200160,122898,0,None,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,22,74,BINANCIAL FREEDOM SENIOR FUNDING CORP,9/28/2007,10/25/2007,None,None,None,None,None,...,None,T,None,None,None,42000,27678,0,35221,02
74,22,75,FINANCIAL FREEDOM SENIOR FUNDING CORP,8/20/2007,9/24/2007,None,None,None,None,None,...,1.03,N,T,None,None,162000,105462,0,36305,02
75,22,76,JAMES B NUTTER AND COMPANY,8/3/2007,10/17/2007,None,None,None,None,None,...,1.5,B,N,T,None,96000,55872,0,36467,02
76,22,77,“JAMES B_NUTTER AND COMPANY,8/6/2007,10/17/2007,None,None,None,None,None,...,1.5,T,None,None,None,75000,41325,0,None,02


# Loop

In [118]:
# ===================== CONFIG =====================
PDF_DIR = r'C:\pdf2csv\extracted_files\recovered'
CHECKPOINT_FILE = r'C:\pdf2csv\checkpoint.pkl'
CHECKPOINT_EVERY = 100
WORKERS = max(mp.cpu_count() - 1, 1)

# ===================== COLUMNS =====================
target_columns = [
    'source_pdf', 'row_in_pdf',
    'lender', 'clsing_dt', 'endrsmt_dt', 'tmntn_dt', 'Borr_Age',
    'Coborr_Age', 'Borr_Cnt', 'es_status', 'int_rt', 'int_rt_10yr',
    'hecm_margin', 'pd_stmln_flg', 'rt_typ', 'arm_indx_typ',
    'arm_prdc_typ', 'max_clm_amt', 'init_prncpl_lmt',
    'hecm_orgntn_fees', 'prop_addr_zip_cd', 'loan_typ'
]

# ===================== PARSER (UNCHANGED) =====================
def robust_parse(line):
    row = [None] * 20

    line = (
        line.replace('”', '"')
            .replace("’", "'")
            .replace('!', '')
            .replace('|', ' ')
    )

    dates = re.findall(r'\d{1,2}/\d{1,2}/\d{2,4}', line)
    decimals = re.findall(r'\d+\.\d+', line)

    clean_text = re.sub(r'\d{1,2}/\d{1,2}/\d{2,4}', ' ', line)
    clean_text = re.sub(r'\d+\.\d+', ' ', clean_text)
    numbers = re.findall(r'\b\d+\b', clean_text)

    lender_match = re.search(r'^(.*?)(?=\d{1,2}/)', line)
    row[0] = lender_match.group(1).strip().strip('"').strip("'") if lender_match else None

    if len(dates) >= 1: row[1] = dates[0]
    if len(dates) >= 2: row[2] = dates[1]
    if len(dates) >= 3: row[3] = dates[2]

    if re.search(r'Termin[a-z]*', line, re.I):
        row[7] = "Terminated"
    elif re.search(r'Active', line, re.I):
        row[7] = "Active"

    if len(decimals) >= 1: row[8] = decimals[0]
    if len(decimals) >= 2: row[9] = decimals[1]
    if len(decimals) >= 3: row[10] = decimals[2]

    if row[3] and row[7]:
        try:
            start = line.index(row[3]) + len(row[3])
            end = line.lower().index(row[7].lower())
            nums = re.findall(r'\b\d+\b', line[start:end])
            if len(nums) == 3:
                row[4], row[5], row[6] = nums
            elif len(nums) == 2:
                row[4], row[6] = nums
        except ValueError:
            pass

    flags = re.findall(r'\b[A-Z]\b', line.upper())
    if len(flags) >= 1: row[11] = flags[0]
    if len(flags) >= 2: row[12] = flags[1]
    if len(flags) >= 3: row[13] = flags[2]

    arm_match = re.search(r'\b\d+[A-Z]\b', line)
    row[14] = arm_match.group(0) if arm_match else None

    large_nums = [n for n in numbers if len(n) > 3]
    if len(large_nums) >= 1: row[15] = large_nums[0]
    if len(large_nums) >= 2: row[16] = large_nums[1]

    row[17] = 0

    if len(numbers) >= 2:
        row[19] = str(numbers[-1]).zfill(2)
        row[18] = numbers[-2] if len(numbers[-2]) == 5 else None

    return row

# ===================== TEXT EXTRACTION =====================
def extract_text(pdf_path):
    try:
        import pdfplumber
        with pdfplumber.open(pdf_path) as pdf:
            text = pdf.pages[0].extract_text()
            if text and len(text.strip()) > 50:
                return text
    except Exception:
        pass

    images = convert_from_path(pdf_path, first_page=1, last_page=1, poppler_path=poppler_bin_path)
    return pytesseract.image_to_string(images[0], config='--oem 3 --psm 6')

# ===================== FILE PROCESSOR =====================
def process_pdf(pdf_path):
    try:
        text = extract_text(pdf_path)
        lines = [l for l in text.split('\n') if len(l.strip()) > 20]
        pdf_name = os.path.basename(pdf_path).replace('.pdf', '')

        rows = []
        for i, line in enumerate(lines, start=1):
            parsed = robust_parse(line)
            rows.append([pdf_name, i] + parsed)

        return rows

    except Exception as e:
        print(f"FAILED: {pdf_path} -> {e}")
        return []

# ===================== CHECKPOINT =====================
def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'rb') as f:
            return pickle.load(f)
    return set(), []

def save_checkpoint(done_files, all_rows):
    with open(CHECKPOINT_FILE, 'wb') as f:
        pickle.dump((done_files, all_rows), f)

# ===================== MAIN =====================
if __name__ == '__main__':
    pdf_files = glob.glob(os.path.join(PDF_DIR, '*.pdf'))
    done_files, all_rows = load_checkpoint()

    remaining = [p for p in pdf_files if p not in done_files]
    print(f"Remaining PDFs: {len(remaining)}")

    pool = mp.Pool(WORKERS)

    for idx, result in enumerate(pool.imap_unordered(process_pdf, remaining), 1):
        pdf_path = remaining[idx - 1]
        done_files.add(pdf_path)
        all_rows.extend(result)

        if idx % CHECKPOINT_EVERY == 0:
            save_checkpoint(done_files, all_rows)
            print(f"Checkpoint saved at {idx} files")

    pool.close()
    pool.join()

    save_checkpoint(done_files, all_rows)

    master_df = pd.DataFrame(all_rows, columns=target_columns)


Remaining PDFs: 13764


AttributeError: module '__main__' has no attribute '__spec__'

# PostProcessing

In [191]:
final_df = pd.read_csv('master_output.csv')
final_df

,source_pdf,row_in_pdf,lender,clsing_dt,endrsmt_dt,tmntn_dt,Borr_Age,Coborr_Age,Borr_Cnt,es_status,int_rt,int_rt_10yr,hecm_margin,pd_stmln_flg,rt_typ,arm_indx_typ,arm_prdc_typ,max_clm_amt,init_prncpl_lmt,hecm_orgntn_fees,prop_addr_zip_cd,loan_typ
0,100,1,GMFS LLC,12/31/2012,2/25/2013,2/16/2017,64.0,NaN,1.0,Terminated,5.06,5.06,NaN,NaN,NaN,NaN,NaN,117000.0,74529.0,0,36108.0,2.0
1,100,2,ONE REVERSE MORTGAGE LLC,12/15/2012,1/25/2013,NaN,NaN,NaN,NaN,NaN,4.50,4.50,NaN,O,N,R,NaN,200000.0,136200.0,0,36312.0,2.0
2,100,3,SUN WEST MORTGAGE CO_INC,1/11/2013,3/4/2013,NaN,NaN,NaN,NaN,NaN,5.30,5.30,NaN,I,I,A,NaN,130000.0,86710.0,0,36571.0,2.0
3,100,4,“MSR_ASSET VEHICLE LLC,2/20/2013,7/25/2013,NaN,NaN,NaN,NaN,NaN,5.06,5.06,NaN,O,NaN,NaN,NaN,300000.0,207900.0,0,36078.0,2.0
4,100,5,SUN WEST MORTGAGE CO_TNC,1/26/2013,4/22/2013,NaN,NaN,NaN,NaN,NaN,4.99,NaN,NaN,NaN,NaN,NaN,NaN,240000.0,161040.0,0,35966.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964660,6852,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60000.0,3200.0,0,97527.0,2.0
964661,6852,62,[TFINANCE OF AWERTCA REVERSE LLC' 27,3/26/2010,12/9/2016,NaN,NaN,NaN,NaN,NaN,5.56,5.56,NaN,NaN,NaN,NaN,NaN,2719.0,2010.0,0,97471.0,2.0
964662,6852,63,['WELLS FARGO BANK NAT a/aaoi0,3/23/2010,NaN,NaN,NaN,NaN,NaN,Terminated,5.49,NaN,NaN,A,NaN,NaN,NaN,54000.0,92092080.0,0,97266.0,2.0
964663,6852,64,['BANK OF AMERICA NA CuARLOTTE,3/2/2010,NaN,NaN,NaN,NaN,NaN,Terminated,6.25,NaN,NaN,NaN,NaN,NaN,NaN,2010.0,97210.0,0,NaN,97210.0


In [177]:
final_df.isnull().sum()

source_pdf               0
row_in_pdf               0
lender               16976
clsing_dt            56336
endrsmt_dt          109649
tmntn_dt            535282
Borr_Age            643621
Coborr_Age          881008
Borr_Cnt            643621
es_status           479483
int_rt              100557
int_rt_10yr         244402
hecm_margin         607678
pd_stmln_flg        124288
rt_typ              384086
arm_indx_typ        771189
arm_prdc_typ        952770
max_clm_amt          30115
init_prncpl_lmt      95515
hecm_orgntn_fees         0
prop_addr_zip_cd    208924
loan_typ             12648
dtype: int64

In [176]:
# Ensure lender is string (in case of NaNs)
lender_str = final_df['lender'].astype(str)

# Create masks
endorsed_mask = (
    final_df['es_status'].isna() &
    lender_str.str.contains(r'endorsed', case=False, regex=True)
)

terminated_mask = (
    final_df['es_status'].isna() &
    lender_str.str.contains(r'terminated', case=False, regex=True)
)

# Assign values
final_df.loc[endorsed_mask, 'es_status'] = 'Endorsed'
final_df.loc[terminated_mask, 'es_status'] = 'Terminated'

In [178]:
# Sorts the DataFrame based on the length of the 'lender' column in descending order
final_df.sort_values(by='lender', key=lambda x: x.str.len(), ascending=False)

,source_pdf,row_in_pdf,lender,clsing_dt,endrsmt_dt,tmntn_dt,Borr_Age,Coborr_Age,Borr_Cnt,es_status,int_rt,int_rt_10yr,hecm_margin,pd_stmln_flg,rt_typ,arm_indx_typ,arm_prdc_typ,max_clm_amt,init_prncpl_lmt,hecm_orgntn_fees,prop_addr_zip_cd,loan_typ
59499,10798,60,[GENERATION MORTGAGE ComPANY' a/ayaoial a/ii...,NaN,NaN,NaN,NaN,NaN,NaN,Terminated,2.204,NaN,NaN,A,A,A,NaN,4805.0,575000.0,0,NaN,2.0
14562,10193,11,"[METLIFE BANK, NATIONAL Assocration' a/avaou...",NaN,NaN,NaN,NaN,NaN,NaN,Terminated,5.030,NaN,NaN,A,I,NaN,NaN,90000.0,23452.0,0,23452.0,2.0
876929,5671,31,NATLONNIDE EQUITIES CoRPoRATION' go/ov2o14] ...,NaN,NaN,NaN,NaN,NaN,NaN,Endorsed,2.903,NaN,NaN,G,A,NaN,NaN,565000.0,64425.0,0,NaN,1762.0
65275,10877,29,['REVERSE MORTGAGE FUNDING LLC' t2tay20i7 i...,NaN,NaN,NaN,NaN,NaN,NaN,Terminated,4.886,NaN,NaN,I,A,A,NaN,98022.0,NaN,0,98022.0,2.0
42632,10570,33,[FINANCIAL FREEDOM SENIOR FUNDING Corp' ___ _t...,NaN,NaN,NaN,NaN,NaN,NaN,Terminated,NaN,NaN,NaN,E,A,A,NaN,2007.0,2007.0,0,98270.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964608,6852,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.560,5.56,NaN,S,NaN,NaN,NaN,97816.0,NaN,0,NaN,97816.0
964620,6852,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.495,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,0,NaN,2.0
964625,6852,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,T,A,NaN,2979.0,28000.0,0,97230.0,2.0
964650,6852,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,P,NaN,NaN,327000.0,97401.0,0,97401.0,2.0


In [168]:
list(final_df['lender'])[0]

'[GENERATION MORTGAGE ComPANY\'   a/ayaoial a/ii/aoia a/a/goni  ta 75]   "Terminated\'   2.204] 4805] as [nh tw ams 575000] 33z925[ 37'

In [172]:
final_df.columns

Index(['source_pdf', 'row_in_pdf', 'lender', 'clsing_dt', 'endrsmt_dt',
       'tmntn_dt', 'Borr_Age', 'Coborr_Age', 'Borr_Cnt', 'es_status', 'int_rt',
       'int_rt_10yr', 'hecm_margin', 'pd_stmln_flg', 'rt_typ', 'arm_indx_typ',
       'arm_prdc_typ', 'max_clm_amt', 'init_prncpl_lmt', 'hecm_orgntn_fees',
       'prop_addr_zip_cd', 'loan_typ'],
      dtype='object')

## Lender

In [ ]:
# Total non unique values in the column lender
final_df['lender'].nunique(dropna=False)

48399

In [ ]:
# Remove all the special characters and spaces from the beginning or the end
final_df['lender'] = final_df['lender'].str.replace(r'^[^a-zA-Z0-9]+|[^a-zA-Z0-9]+$', '', regex=True)

In [127]:
final_df['lender'].nunique(dropna=False)

42787

In [ ]:
# Turn all the characters to UPPER STRING
final_df['lender'] = final_df['lender'].str.upper()

In [129]:
final_df['lender'].nunique(dropna=False)

40299

### Most Frequent

In [161]:
# Returns a Series where the index is the Lender and the value is the count
final_df['lender'].value_counts()

lender
FINANCIAL FREEDOM SENIOR FUNDING CORP           142416
WELLS FARGO BANK NA                             127062
AMERICAN ADVISORS GROUP                          80479
BANK OF AMERICA NA CHARLOTTE                     34905
SEATTLE MORTGAGE COMPANY                         32436
                                                 ...  
SUINIT FUNDING TNC                                   1
J_AND_R LENDING                                      1
GENERATION MORTGAGE COWPANY'   @/TDOOIZ              1
MOVEMENT WORTGAGE LLC'   5 /A/O0I2                   1
BANK OF AMERICA NA CHARLOTTE'   A/TOVAOI0  3         1
Name: count, Length: 29765, dtype: int64

#### WELLS FARGO BANK NA

In [ ]:
# Returns unique values that contain a particular string
# 1. Define your search variable
search_string = "FARGO"

# 2. Filter the column and get unique values
# na=False handles rows with missing data so the code doesn't error out
list(final_df[final_df['lender'].str.contains(search_string, na=False)]['lender'].unique())

In [145]:
# Use .loc to find rows where 'lender' contains 'GMFS' and update the 'lender' column
final_df.loc[final_df['lender'].str.contains('FARGO BANK', na=False), 'lender'] = 'WELLS FARGO BANK NA'

In [147]:
final_df['lender'].nunique(dropna=False)

35206

#### AMERICAN ADVISORS GROUP

In [151]:
# Returns unique values that contain a particular string
# 1. Define your search variable
search_string = "ADVISORS"

# 2. Filter the column and get unique values
# na=False handles rows with missing data so the code doesn't error out
list(final_df[final_df['lender'].str.contains(search_string, na=False)]['lender'].unique())

['AMERICAN ADVISORS GROUP',
 'AMERICAN ADVISORS GROUP\'   A/TAZYAONO] GENNI]   WA]   NDORSED "  5.625] 5.625] ANY TE G60',
 "AMERICAN ADVISORS GROUP'   A/TZAON0",
 'TAMERICAN ADVISORS GROUP™   87',
 "AMERICAN ADVISORS GROUP' _   2010",
 "AMERICAN ADVISORS GROUP'   T0IGY20N0",
 'TAMBRICAN ADVISORS GROUP',
 "TAMERICAN ADVISORS GROUP'   2Z/2Z0NN",
 "AMERICAN ADVISORS GROUP'   TI2Z0NI",
 "AMERICAN ADVISORS GROUP'   AON",
 "TAMERICAN ADVISORS GROUP'   A",
 'TAMERICAN ADVISORS GROUP   87',
 "AMERICAN ADVISORS GROUP'   A5 FO0NI",
 "AMERICAN ADVISORS GROUP'   A/ZA0NI",
 'TAMERICAN ADVISORS GROUP AVZI/AONN',
 "AMERICAN ADVISORS GROUP' 5 /TFA0NI",
 "AMERICAN ADVISORS GROUP'   AZAR",
 "AMERICAN ADVISORS GROUP'   A/ZZ/A0NI  5 /A",
 "AMERICAN ADVISORS GROUP'   W",
 'TAMERICAN ADVISORS GROUT   Z',
 "AMERICAN ADVISORS GROUP' 6A FO0NI",
 "AMERICAN ADVISORS GROUP'   ATIZ0NI",
 'TAMERICAN ADVISORS GROUP',
 "AMERICAN ADVISORS GROUP' _   G4Y2OUI  G/A",
 'AMERICAN ADVISORS GROUT',
 "AMERICAN ADVISORS GROUP

In [152]:
# Use .loc to find rows where 'lender' contains 'GMFS' and update the 'lender' column
final_df.loc[final_df['lender'].str.contains('ADVISORS', na=False), 'lender'] = 'AMERICAN ADVISORS GROUP'

In [154]:
final_df['lender'].nunique(dropna=False)

30189

#### FINANCIAL FREEDOM SENIOR FUNDING CORP

In [158]:
# Returns unique values that contain a particular string
search_string = "OR FUN"

# na=False handles rows with missing data so the code doesn't error out
list(final_df[final_df['lender'].str.contains(search_string, na=False)]['lender'].unique())

['PINANCIAL FREEDOM SENIOR FUNDING CORP',
 'FINANCIAL FREEDOM SENIOR FUNDING CORP',
 'PINANCTAL FREEDOM SENTOR FUNDING CORP',
 'RINANCTAL FREEDOM SENTOR FUNDING CORP',
 'BINANCIAL FREEDOM SENIOR FUNDING CORP',
 'RINANCIAL FREEDOM SENIOR FUNDING CORP',
 'PINANCIAL FREEDOM SENTOR FUNDING CORP',
 'TRINANCIAL FREEDOM SENIOR FUNDING CORP',
 'FINANCTAL FREEDOM SENIOR FUNDING CORP',
 'PINANCTAL FREEDOM SENIOR FUNDING CORP',
 "RINANCTAL FREEDOM SENIOR FUNDING CORP'   AZ/A/2006",
 'RINANCTAL FREEDOM SENIOR FUNDING CORP',
 "FINANCIAL FREEDOM SENIOR FUNDING CORP' _____I",
 'TRINANCTAL FREEDOM SENIOR FUNDING CORP',
 'FINANCIAL FREEDOM SENTOR FUNDING CORP',
 'FINANCIAL FREEDOM SENJOR FUNDING CORP',
 'BINANCTAL FREEDOM SENIOR FUNDING CORP',
 'FINANCIAL FREEDON SENIOR FUNDING CORP',
 'TFINANCIAL FREEDOM SENIOR FUNDING CORP',
 'TBINANCIAL FREEDOM SENIOR FUNDING CORP',
 'TFINANCTAL FREEDOM SENIOR FUNDING CORP',
 'TFINANCIAL FREEDOM SENTOR FUNDING CORP',
 "RINANCTAL FREEDOM SENIOR FUNDING CORP'   A/AZ/2

In [159]:
# Use .loc to find rows where 'lender' contains 'GMFS' and update the 'lender' column
final_df.loc[final_df['lender'].str.contains('OR FUN', na=False), 'lender'] = 'FINANCIAL FREEDOM SENIOR FUNDING CORP'

In [160]:
final_df['lender'].nunique(dropna=False)

29766